# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [4]:
# the driver is always the site where you are
# The xptah of the controller search //*[@id="inputdrs"] is the same one as the additional search
# Then, I copied the xpath of the font style inside the td //*[@id="content"]/table[3]/tbody/tr[3]/td[1]/font[1]
controller = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", controller)
controller.send_keys('3503598')






In [5]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [6]:
change_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
change_date.send_keys('1/1/1995')

In [7]:
report_type = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
report_type.click()

In [8]:
get_report_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
get_report_button.click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [9]:
reports = []

rows = driver.find_elements_by_css_selector('tr.drsviols')

for row in rows:
        
    data = {}
    data['violator'] = row.find_elements_by_tag_name('td')[0].text
    data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
    data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
    data['case_no'] = row.find_elements_by_tag_name('td')[3].text
    data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
    data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
    data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
    data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
    data['citation'] = row.find_elements_by_tag_name('td')[8].text
    data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
    data['standard'] = row.find_elements_by_tag_name('td')[10].text
    data['standard_url'] = row.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
    data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
    data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
    data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
    data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text

    reports.append(data)

rows  = driver.find_elements_by_css_selector('tr.drsviols')
reports = []

for row in rows:
    
    cells = row.find_elements_by_tag_name('td')
  
    data = {}
    data['violator'] = row.find_elements_by_tag_name('td')[0].text
    data['contract_id'] = cells[1].text
    data['citation_no'] = cells[2].text
    data['case_no'] = cells[3].text
    data['date_issues'] = cells[4].text
    data['final_order_date'] = cells[5].text
    data['section_of_act'] = cells[6].text
    data['date_terminated'] = cells[7].text
    data['citation'] = cells[8].text
    data['s_and_s'] = cells[9].text
    data['standard'] = cells[10].text
    data['standard_url'] = cells[10].find_element_by_tag_name('a').get_attribute('href')
    data['proposed_penalty'] = cells[11].text
    data['citation_status'] = cells[12].text
    data['current_penalty'] = cells[13].text
    data['amount_paid'] = cells[14].text


    reports.append(data)

In [10]:
df_violations = pd.DataFrame(reports)
df_violations

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
5,100.00,000383840,C,8790964,Closed,,100.00,4/13/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
6,100.00,000383840,C,8790962,Closed,,100.00,4/10/2015,4/10/2015,7/17/2015,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
7,873.00,000383840,C,8790963,Closed,,873.00,4/10/2015,4/13/2015,7/17/2015,873.00,Y,104(a),56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
8,100.00,000345066,C,8694458,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt
9,100.00,000345066,C,8694457,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt


### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [11]:
df_violations.to_csv("3503598-violations.csv", index=False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [12]:
# After trying so many times, I included all the code needed to scrape the site in one cell
# as I went mad going back and running several times the same snippets of code

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

driver = webdriver.Chrome()

driver.get("https://arlweb.msha.gov/drs/drshome.htm")

controller = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", controller)
controller.send_keys('3503598')

search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

change_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
change_date.send_keys('1/1/1995')

report_type = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
report_type.click()

get_report_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
get_report_button.click()

reports = []

rows = driver.find_elements_by_css_selector('tr.drsviols')

for row in rows:
        
    data = {}
    data['violator'] = row.find_elements_by_tag_name('td')[0].text
    data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
    data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
    data['case_no'] = row.find_elements_by_tag_name('td')[3].text
    data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
    data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
    data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
    data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
    data['citation'] = row.find_elements_by_tag_name('td')[8].text
    data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
    data['standard'] = row.find_elements_by_tag_name('td')[10].text
    data['standard_url'] = row.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
    data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
    data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
    data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
    data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text

    reports.append(data)
    
df_violations = pd.DataFrame(reports)
df_violations.head()

df_violations.to_csv("3503598-violations.csv", index=False)

In [13]:
# Checking everything is fine

df = pd.read_csv("3503598-violations.csv")
df.head()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.0,440809,C,8992368,Closed,,116.0,5/2/2017,5/2/2017,7/23/2017,116.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.0,435717,C,8992214,Closed,,116.0,2/7/2017,2/8/2017,5/21/2017,116.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.0,398910,C,8872462,Closed,,100.0,10/21/2015,10/27/2015,1/13/2016,100.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.0,398910,C,8872461,Closed,,100.0,10/21/2015,10/27/2015,1/13/2016,100.0,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.0,383840,C,8790965,Closed,,100.0,4/15/2015,4/15/2015,7/17/2015,100.0,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [14]:
df = pd.read_csv("mines-subset.csv")

In [15]:
df.head()

,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [16]:
df = pd.read_csv("mines-subset.csv", dtype='str')
df.head()

,id
0,4104757
1,0801306
2,3609931


### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

def scrape_mines_subset(id):

    driver = webdriver.Chrome()

    driver.get("https://arlweb.msha.gov/drs/drshome.htm")

    controller = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]/input[1]')
    driver.execute_script("arguments[0].scrollIntoView(true)", controller)
    controller.send_keys(id)

    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()

    change_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
    change_date.send_keys('1/1/1995')

    report_type = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    report_type.click()

    get_report_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    get_report_button.click()

    reports = []

    rows = driver.find_elements_by_css_selector('tr.drsviols')

    for row in rows:
        
        data = {}
        data['violator'] = row.find_elements_by_tag_name('td')[0].text
        data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
        data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
        data['case_no'] = row.find_elements_by_tag_name('td')[3].text
        data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
        data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
        data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
        data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
        data['citation'] = row.find_elements_by_tag_name('td')[8].text
        data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
        
        try: 
            
            data['standard'] = row.find_elements_by_tag_name('td')[10].text
            data['standard_url'] = row.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
            data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
            data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
            data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
            data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
        
        except:

            data['standard'] = 'N/A' # Not assesed or Non-Assessable or N/A
            data['standard_url'] = 'N/A'
            data['proposed_penalty'] = 'N/A'
            data['citation_status'] = 'N/A'
            data['current_penalty'] = 'N/A'
            data['amount_paid'] = 'N/A'
            
        reports.append(data)
    
    df_violations = pd.DataFrame(reports)
    df_violations.head()
    
    df_violations.to_csv("{}-violations.csv".format(id), index=False)

In [18]:
# Don't really understand the Nones but it is working. Ask this!
df['id'].apply(scrape_mines_subset)

0    None
1    None
2    None
Name: id, dtype: object

# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [19]:
df_all_mines = pd.read_csv("mines.csv", dtype='str')
df_all_mines

,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

def scrape_mines_subset(id):

    driver = webdriver.Chrome()

    driver.get("https://arlweb.msha.gov/drs/drshome.htm")

    controller = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]/input[1]')
    driver.execute_script("arguments[0].scrollIntoView(true)", controller)
    controller.send_keys(id)

    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()

    change_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
    change_date.send_keys('1/1/1995')

    report_type = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    report_type.click()

    get_report_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    get_report_button.click()

    reports = []

    rows = driver.find_elements_by_css_selector('tr.drsviols')

    for row in rows:
        
        data = {}
        data['violator'] = row.find_elements_by_tag_name('td')[0].text
        data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
        data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
        data['case_no'] = row.find_elements_by_tag_name('td')[3].text
        data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
        data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
        data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
        data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
        data['citation'] = row.find_elements_by_tag_name('td')[8].text
        data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
        
        try: 
            
            data['standard'] = row.find_elements_by_tag_name('td')[10].text
            data['standard_url'] = row.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
            data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
            data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
            data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
            data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
        
        except:

            data['standard'] = 'N/A' # Not assesed or Non-Assessable or N/A
            data['standard_url'] = 'N/A'
            data['proposed_penalty'] = 'N/A'
            data['citation_status'] = 'N/A'
            data['current_penalty'] = 'N/A'
            data['amount_paid'] = 'N/A'
            
        reports.append(data)
        
        # I included return to check that everything is going well while scraping.
        # It returns the first row of each .csv
        return pd.Series(data)
    
    df_violations = pd.DataFrame(reports)
    df_violations.head()
    
    df_violations.to_csv("{}-violations.csv".format(id), index=False)

In [25]:
# Is there a way to scrape it without opening a new browser window each time?
df_all_mines['id'].apply(scrape_mines_subset)

,violator,contract_id,citation_no,case_no,date_issues,final_order_date,section_of_act,date_terminated,citation,s_and_s,standard,standard_url,proposed_penalty,citation_status,current_penalty,amount_paid
0,Newberg Rock & Dirt,,8992368,000440809,5/2/2017,7/23/2017,104(a),5/2/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,116.00,Closed,116.00,116.00
1,AM Dirtworks & Aggregate Sales,,8933510,000426827,11/1/2016,1/20/2017,104(a),1/11/2017,C,N,56.1000,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,114.00,Delinquent,114.00,0.00
2,Dirt Company,,8601716,000423303,10/1/2016,12/16/2016,104(a),10/28/2016,C,N,56.14103(a),http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,114.00,Closed,114.00,114.00
3,Sunshine Reclamation Inc.,,8657165,000291395,5/1/2012,7/25/2012,104(a),5/1/2012,C,Y,56.9300(b),http://www.gpo.gov/fdsys/pkg/CFR-2012-title30-...,108.00,Closed,108.00,108.00
4,Dirt Doctor Inc,,8946233,000422613,8/31/2016,11/25/2016,104(a),8/31/2016,C,Y,56.18002(a),http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,124.00,Delinquent,124.00,0.00
5,Dirt Works,,8778047,000374480,10/14/2014,3/27/2015,104(a),11/4/2014,C,N,56.18010,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,162.00,Closed,162.00,162.00
6,"Holley Dirt Company, Inc",,9421625,,5/15/2018,,104(a),5/15/2018,C,Y,N/A,N/A,N/A,N/A,N/A,N/A
7,Krueger Brothers Gravel & Dirt,,8750964,000361866,8/12/2014,10/22/2014,104(a),9/2/2014,C,N,56.18010,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.00,Closed,100.00,100.00
8,M R Dirt,,8922247,000398559,10/29/2015,1/10/2016,104(a),10/29/2015,C,N,56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,100.00,Closed,100.00,100.00
9,M.R. Dirt Inc.,,9317668,000442816,5/11/2017,8/19/2017,104(a),5/11/2017,C,N,50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,116.00,Closed,116.00,116.00


In [24]:
# The .csv files are in the folder!